In [1]:
from qtpg.team import Team
from qtpg.learner import Learner
from qtpg.program import Program
from qtpg.rule import Rule
from qtpg.figure13 import Figure13
from qtpg.search_manager import SearchManager
import numpy as np
import uuid
import copy

In [2]:
# env params
memorySize = 20
legalMove = 0.1
illegalMove = -0.01
outOfBounds = -0.01
memoryRepeat = 0 #-0.01
goalReached = 1000

env = Figure13(5, 5, (2, 4), (0, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)

In [3]:
adam_and_eve = []
env.reset()
init_team = Team(uuid.uuid4(), 0, 0, 0, 0, 0)
# init_rule = Rule(uuid.uuid4(), [0, 0, 0, 0], 0, 0)
# init_learner = Learner(uuid.uuid4(), init_rule)
# (init_team.learners).append(init_learner)
# init_team.mostRecent = init_learner
adam_and_eve.append(init_team)

for adam in adam_and_eve:
    print(adam.learners)
    print('------')
    adam.init_search(env, 2)
    print(adam.learners)
    for learner in adam.learners:
        print(learner.program.rule.action_set)
        print(learner.program.rule.region)

[]
------
2
[0, 0, 0, 4]


In [4]:
numGens = 50
maxTeamPool = 5
# init search manager and starting teams
searchManager = SearchManager(maxTeamPool)

# init_team = Team(uuid.uuid4(), 0, 0, 0, 0, 0)
# init_rule = Rule(uuid.uuid4(), [0, 0, 0, 0], 0, 0)
# init_learner = Learner(uuid.uuid4(), init_rule)
# (init_team.learners).append(init_learner)
# init_team.mostRecent = init_learner
# (searchManager.teamPool).append(init_team)

# init teams
adam_and_eve = []
env.reset()
init_team = Team(uuid.uuid4(), 0, 0, 0, 0, 0)
init_team.init_search(env, 2)
adam_and_eve.append(init_team)

# print('-----init pop start-----')
# for team in searchManager.teamPool:
#     print(f'Team id: {team.id}')
#     print('Learners:')
#     for learner in team.learners:
#         print(learner.id)
# print('-----init pop end-----')

for gen in range(numGens):
    toEvaluateAfterGen = []
    print(f'>>>>>>>>>>>>>>>>>>>>>>>Gen {gen+1} start!>>>>>>>>>>>>>>>>>>>>>>>')
    # get it started
    if gen == 0:
        for adam in adam_and_eve:
            win = adam.search(env)
            data = [adam, win]
            searchManager.evaluate_team(data)
    else:
        for parent in searchManager.teamPool:
            # only run if the team has not yet won
            if parent not in searchManager.winners:
                print(f'Parent: {parent.id}')
                env.current_state = parent.start_state

                # make offspring from parent, then let that offspring make a new rule
                child = copy.deepcopy(parent)
                child.id = uuid.uuid4()
                win = child.search(env)
                print(f'Child: {child.id}')
                print('Start Child Regions --------')
                for learner in child.learners:
                    print(learner.program.rule.region)
                print('End Child Regions --------')
                print(f'Child Fitness: {child.fitness}') 
                toEvaluateAfterGen.append([child, win])
            else:
                print(f'Skipping Team: {parent.id}')
    for data in toEvaluateAfterGen:
        searchManager.evaluate_team(data)
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print(f'End of Gen {gen+1} Regions')
    for i in range(len(searchManager.teamPool)):
        print(f'Team: {i+1} --> {searchManager.teamPool[i].fitness}')
        for learner in searchManager.teamPool[i].learners:
            print(learner.program.rule.region)
    print(f'>>>>>>>>>>>>>>>>>>>>>>>Gen {gen+1} end!>>>>>>>>>>>>>>>>>>>>>>>')
for team in searchManager.winners:
    print('$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$')
    print(team.fitness)
    print('Learners:')
    for learner in team.learners:
        print(f'Region: {learner.program.rule.region} --> Action: {learner.program.rule.action_set}')
    print('$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$')

>>>>>>>>>>>>>>>>>>>>>>>Gen 1 start!>>>>>>>>>>>>>>>>>>>>>>>
Backtrackregioncehck: [0, 0, 0, 4]
Backtrackregioncehck: [0, 0, 1, 4]
Backtrackregioncehck: [0, 0, 2, 4]
Backtrackregioncehck: [0, 0, 3, 4]
Backtrackregioncehck: [0, 0, 0, 4]
Backtrackregioncehck: [0, 0, 1, 4]
Backtrackregioncehck: [0, 0, 2, 4]
Backtrackregioncehck: [0, 0, 3, 4]
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
End of Gen 1 Regions
Team: 1 --> 0.31000000000000005
[0, 0, 4, 4]
[1, 4, 0, 0]
>>>>>>>>>>>>>>>>>>>>>>>Gen 1 end!>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>>>>>>>>>Gen 2 start!>>>>>>>>>>>>>>>>>>>>>>>
Parent: ff2b9e84-9a4d-453e-967d-3cd2d7b27a97
Child: 9db1a5ec-9985-4cd3-a2e1-63101e17f085
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 1, 4]
End Child Regions --------
Child Fitness: 0.7100000000000001
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
End of Gen 2 Regions
Team: 1 --> 0.31000000000000005
[0, 0, 4, 4]
[1, 4, 0, 0]
Team: 2 --> 0.7100000000000001
[0, 0, 4

[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 2]
[1, 2, 0, 3]
[0, 4, 1, 2]
[1, 0, 4, 4]
End Child Regions --------
Child Fitness: 2.7100000000000004
Parent: 56cecbc3-76db-4569-a80a-93f5af2a31ba
Child: bc0bce31-727c-4a72-bac1-bdebb5e159f2
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 3]
End Child Regions --------
Child Fitness: 2.7300000000000004
Parent: 0cca7d9c-e59f-4a12-82fb-2724575f97f4
Child: 5f0ec45f-f2f2-498e-b0b2-2170de8f734a
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 2]
[1, 0, 4, 4]
[0, 3, 0, 0]
End Child Regions --------
Child Fitness: 2.4200000000000004
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
End of Gen 15 Regions
Team: 1 --> 2

Child: 82a12220-0904-4869-a3e7-582d1ad3ad52
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 3]
End Child Regions --------
Child Fitness: 3.7300000000000004
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
End of Gen 22 Regions
Team: 1 --> 3.9300000000000006
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 3]
[0, 4, 1, 2]
Team: 2 --> 3.9300000000000006
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0

[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 3]
[0, 4, 1, 2]
[1, 0, 4, 4]
>>>>>>>>>>>>>>>>>>>>>>>Gen 27 end!>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>>>>>>>>>Gen 28 start!>>>>>>>>>>>>>>>>>>>>>>>
Parent: 454b6fcb-e748-4586-93a3-dfe829c6dbe8
Backtrackregioncehck: [1, 0, 0, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Child: 53ea1b49-1751-4346-a076-5c58ae3c4229
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1,

Backtrackregioncehck: [1, 0, 0, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Child: b4ff896a-3133-488f-95cb-dc732829e416
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 3]
[0, 4, 1, 2]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 2, 4]
[0, 1, 0, 0]
End Child Regions --------
Child Fitness: 4.460000000000002
Parent: 5a103433-328b-4ed5-81a7-f102a29b65a8
Backtrackregioncehck: [1, 0, 0, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregionc

[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 3]
[0, 4, 1, 2]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
Team: 5 --> 4.860000000000002
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 3]
[0, 4, 1, 2]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
>>>>>>>>>>>>>>>>>>>>>>>Gen 35 end!>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>>>>>>>>>Gen 36 start!>>>>>>>>>>>>>>>>>>>>>>>
Parent: a9ddc520-71af-4b31-b493-4b659d0b7abb
Backtrackregioncehck: [0, 4, 0, 3]
Backtrackregioncehck: [0, 4, 0, 2]
Backtrackregioncehck: [0, 4, 0, 1]
Child: c5d8f175-7ee5-4bda-82c2-28087114da57
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4

[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 3]
[0, 4, 1, 2]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
End Child Regions --------
Child Fitness: 5.480000000000003
Parent: bba37c38-7d65-4d96-8a91-70166d1db764
Child: ba272121-f8e0-4c41-936a-4d8e6646bfb0
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 3]
[0, 4, 1, 2]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 3]
End Child Regions --------
Child Fitness: 5.8800000000000034
Parent: c5d8f175-7ee5-4bda-82c2-28087114da57
Backtrackregioncehck: [1, 0, 0, 4]
Backtrackregioncehck: [1, 0, 1,

[0, 4, 1, 2]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 3]
Team: 3 --> 5.8800000000000034
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 3]
[0, 4, 1, 2]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 3]
Team: 4 --> 6.190000000000004
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 3]
[0, 4, 1, 2]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0,

[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 3]
[0, 4, 1, 2]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 3]
Team: 3 --> 6.500000000000004
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 3]
[0, 4, 1, 2]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 3]
Team: 4 --> 6.190000000000004
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 

Backtrackregioncehck: [0, 4, 0, 3]
Backtrackregioncehck: [0, 4, 0, 2]
Backtrackregioncehck: [0, 4, 0, 1]
Child: 19b75d43-a7bd-466f-8a12-f15c5a6647b0
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 0, 0, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 3]
[1, 4, 0, 0]
[0, 0, 1, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 3]
[0, 4, 1, 2]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
End Child Regions --------
Child Fitness: 6.410000000000005
Parent: 135b1540-a2ca-40d0-b8b9-013cd16d74a1
Backtrackregioncehck: [1, 0, 1, 1]
Backtrackregioncehck: [1, 0, 1, 1]
Backtrackregioncehck: [1, 0, 1, 1]
Backtrackregioncehck: [1, 0, 1, 1]
Backtrackregioncehck: [1, 0, 1, 1]
Backtrackregioncehck: [1, 0, 1, 1]
Backtrackregi

In [5]:
for team in searchManager.winners:
    print('$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$')
    print(team.fitness)
    print('Learners:')
    for learner in team.learners:
        print(f'Region: {learner.program.rule.region} --> Action: {learner.program.rule.action_set}')
    print('$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$')

$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$
1003.13
Learners:
Region: [0, 0, 4, 4] --> Action: 2
Region: [1, 4, 0, 0] --> Action: 1
Region: [0, 0, 1, 4] --> Action: 3
Region: [1, 0, 0, 0] --> Action: 1
Region: [0, 0, 0, 3] --> Action: 2
Region: [1, 4, 0, 0] --> Action: 1
Region: [0, 0, 1, 4] --> Action: 3
Region: [1, 0, 0, 0] --> Action: 1
Region: [0, 0, 0, 3] --> Action: 2
Region: [1, 4, 0, 0] --> Action: 1
Region: [0, 0, 1, 4] --> Action: 3
Region: [1, 0, 0, 0] --> Action: 0
Region: [0, 1, 0, 1] --> Action: 2
Region: [1, 2, 1, 3] --> Action: 0
Region: [0, 4, 2, 3] --> Action: 2
Region: [1, 4, 2, 4] --> Action: 1
$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$
1003.13
Learners:
Region: [0, 0, 4, 4] --> Action: 2
Region: [1, 4, 0, 0] --> Action: 1
Region: [0, 0, 1, 4] --> Action: 3
Region: [1, 0, 0, 0] --> Action: 1
Region: [0, 0, 0, 3] --> Action: 2
Region: [1, 4, 0, 0] --> Action: 1
Region: [0, 0, 1, 4] --> Action: 3
Regio

Region: [1, 0, 4, 4] --> Action: 0
Region: [0, 4, 0, 3] --> Action: 2
Region: [1, 4, 2, 4] --> Action: 1
$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$
1007.0100000000001
Learners:
Region: [0, 0, 4, 4] --> Action: 2
Region: [1, 4, 0, 0] --> Action: 1
Region: [0, 0, 1, 4] --> Action: 3
Region: [1, 0, 0, 0] --> Action: 1
Region: [0, 0, 0, 3] --> Action: 2
Region: [1, 4, 0, 0] --> Action: 1
Region: [0, 0, 1, 4] --> Action: 3
Region: [1, 0, 0, 0] --> Action: 1
Region: [0, 0, 0, 3] --> Action: 2
Region: [1, 4, 0, 0] --> Action: 1
Region: [0, 0, 1, 4] --> Action: 3
Region: [1, 0, 0, 0] --> Action: 0
Region: [0, 1, 0, 1] --> Action: 2
Region: [1, 2, 1, 1] --> Action: 1
Region: [0, 0, 2, 3] --> Action: 2
Region: [1, 4, 0, 0] --> Action: 1
Region: [0, 0, 1, 4] --> Action: 3
Region: [1, 0, 0, 0] --> Action: 0
Region: [0, 1, 0, 1] --> Action: 2
Region: [1, 2, 1, 3] --> Action: 0
Region: [0, 4, 1, 2] --> Action: 3
Region: [1, 0, 4, 4] --> Acti